### Important: run the blocks one by one and not with "run_all" command ###
Imports from the main class

In [ ]:
from focus_differentiator import loadImgTrainModel, relabelMasks, saveModelSegmentationsAndProps, loadNd2SaveTiff, loadModelSegmentations, DifferentiateFocusLayer, createFocusMasks, loadFocusMasks, useBrushToAddAndDrop, relabelMasksConsecutive, getRegionProps, addHeterocystColumn
import os
import numpy as np

Constant parameters. Details in comment before the constant

In [ ]:
FOV = 0
HYB = 0
# Phase image directory. The file name is "fov_x_hyb_y.nd2" where x and y are int numbers
DATA_DIR = r'Z:\sandbox\for_vadim\nostoc_auto_110924'
FILE_NAME = f'fov_{FOV}_hyb_{HYB}'
# Start with sub image and adjust parameters accordingly
IMG_SLICE = [0, -1, 0, -1]
#IMG_SLICE = [300, 600, 300, 600]
# If you wish to load the focus masks, this is their path
FOCUS_MASKS_PATH = rf'C:\Users\LITVINOV\PycharmProjects\anabaenaSeg\fov_{FOV}_hyb_{HYB}\{FILE_NAME}.seg.npy'
# The minimum values for filtering masks by CellPose probability. Combined = (mean+median)/2, SNR = mean/SD
MIN_COMBINED_PROBA = 4.25
MIN_MEDIAN_PROBA = 4.25
MIN_SNR_PROBA = 4.25
# Radius to search other masks in different z slices
RADIUS_FROM_CENTROID = 2
# Determine Z and X,Y span of each cell by looking at the image (for the sliced region)
Z_SPAN = 3
XY_SPAN = 3
# The operator for the 3 probability constraints. If "and" is too strict, use "or"
#OPERATOR = 'and'
OPERATOR = 'or'
# If it is the first run, assign all False. The program will create the CellPose output masks, will differentiate the focus masks out of them, and will calculate the region image properties for each focus mask (area, perimeter, centroid, etc.)
LOAD_MODEL_OUTPUT_MASKS = False
LOAD_ALGORITHM_FOCUS_MASKS = False
LOAD_REGION_PROPS = False

1.  1. If LOAD_MODEL_OUTPUT_MASKS is False, load the .nd2 file, and predict segmentation masks with CellPose.
    2. Relabel the masks, such that each mask in each z slice will have unique identifier.
    3. Save model segmentation masks as .npy file
2.  1. if LOAD_MODEL_OUTPUT_MASKS is True, change location to the current Field Of View directory where you want to save the focus masks
    2. Load the .nd2 file
    3. load CellPose model saved segmentations masks

In [ ]:
cwd = os.getcwd()
target_dir = os.path.join(cwd, FILE_NAME)
os.makedirs(target_dir, exist_ok=True)
if not LOAD_MODEL_OUTPUT_MASKS:
    image, original_masks, flows, styles, probabilities = loadImgTrainModel(FOV, HYB, DATA_DIR, IMG_SLICE)
    masks_relab = relabelMasks(original_masks)
    saveModelSegmentationsAndProps(masks_relab, None, None, probabilities, FOV, HYB, to_final_folder=False, dir_name=FILE_NAME)
else:
    #os.chdir(f'C:\\Users\\LITVINOV\\PycharmProjects\\anabaenaSeg\\{FILE_NAME}')
    image = loadNd2SaveTiff(DATA_DIR, FOV, HYB, IMG_SLICE)
    masks_relab, probabilities = loadModelSegmentations(FILE_NAME, FOV)

Take only the phase for differentiating the focus layer. If LOAD_ALGORITHM_FOCUS_MASKS is False, create the data structures and differentiate the focus masks from all the masks outputted by CellPose. If it is TRUE, load it

In [ ]:
img_phase_channel = image[:, 0, :, :]
if not LOAD_ALGORITHM_FOCUS_MASKS:
    masks_copy = np.array(masks_relab, copy=True)
    labels_to_keep_main, filtered_masks_relab = DifferentiateFocusLayer(probabilities, img_phase_channel, masks_copy, Z_SPAN, XY_SPAN, RADIUS_FROM_CENTROID, MIN_SNR_PROBA, MIN_MEDIAN_PROBA, MIN_COMBINED_PROBA, OPERATOR)
    focus_masks = createFocusMasks(masks_relab, labels_to_keep_main)
else:
    focus_masks = loadFocusMasks(FOCUS_MASKS_PATH)

The algorithm may not produce the optimal focus masks for all cells, so manually correct with brush in Napari viewer

In [ ]:
focus_masks = useBrushToAddAndDrop(img_phase_channel, masks_relab, focus_masks)

The masks are filtered, and have their old non-consecutive names. Relabel them

In [ ]:
focus_masks = relabelMasksConsecutive(focus_masks)

Now we need to create region props for the masks (area, centroid, orientation, eccentricity...). Create or Load them if it is already exists

In [ ]:
if LOAD_REGION_PROPS:
    prop_df_merged = getRegionProps(None, file_name=FILE_NAME, load=True)
else:
    prop_df_merged = getRegionProps(focus_masks, file_name=None, load=False)

Manually add heterocyst column to the region props. 1 if heterocyst and 0 is vegetative

In [ ]:
prop_df_merged = addHeterocystColumn(img_phase_channel, focus_masks, prop_df_merged)

Save all the CellPose masks, the focus masks, and the props of the focus masks to the current working directory

In [ ]:
saveModelSegmentationsAndProps(masks_relab, focus_masks, prop_df_merged, proba=None, fov=FOV, hyb=HYB,
                                   to_final_folder=True, dir_name=FILE_NAME)